In [1]:
import os
import random
import time
import wandb
from distutils.util import strtobool
from typing import Callable
import flax
import flax.linen as nn
import gymnasium as gym
import jax
import jax.numpy as jnp
import numpy as np
import optax

from flax.training.train_state import TrainState
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

In [2]:
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"video/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)

        return env

    return thunk

In [3]:
class QNetwork(nn.Module):
    action_dim: int

    @nn.compact
    def __call__(self, x: jnp.ndarray):
        x = nn.Dense(120)(x)
        x = nn.relu(x)
        x = nn.Dense(84)(x)
        x = nn.relu(x)
        x = nn.Dense(self.action_dim)(x)
        # x = nn.relu(x)

        return x


In [4]:
class TrainState(TrainState):
    target_params: flax.core.FrozenDict

In [5]:
def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e)/ duration
    return max(slope * t + start_e, end_e)

In [6]:
def evaluate(model_path: str,
             make_env: Callable,
             env_id: str,
             eval_episodes: int,
             run_name: str,
             Model: nn.Module,
             epsilon: float = 0.05,
             capture_video: bool = True,
             seed=1):
    envs = gym.vector.SyncVectorEnv([make_env(env_id, 0, 0, capture_video, run_name)])
    obs, _ = envs.reset()
    model = Model(action_dim=envs.single_action_space.n)
    q_key = jax.random.PRNGKey(seed)
    params = model.init(q_key, obs)
    with open(model_path, "rb") as f:
        params = flax.serialization.from_bytes(params, f.read())
    model.apply = jax.jit(model.apply)

    episodic_returns = []

    while len(episodic_returns) < eval_episodes:
        if random.random() < epsilon:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            q_values = model.apply(params,obs)
            actions = q_values.argmax(axis=-1)
            actions = jax.device_get(actions)
        next_obs, _, _, _, infos = envs.step(actions)
        if "final_info" in infos:
            for info in infos["final_info"]:
                if "episode" not in info:
                    continue
                print(f"eval_episode={len(episodic_returns)}, episodic_return={info['episode']['r']}")
                episodic_returns += [info["episode"]["r"]]

        obs = next_obs

    return episodic_returns

In [12]:
envs = {"cartPole":"CartPole-v1","acrobot":"Acrobot-v1","mountainCar":"MountainCar-v0"}
n_timesteps = 500000
learning_rate = 0.00025
buffer_size = 10000
gamma = 0.99
target_network_frequency = 500
max_grad_norm = 0.5
batch_size = 128
start_e = 1
end_e = 0.05
exploration_fraction = 0.3
learning_starts = 10000
train_frequency = 10
seed = 1
capture_video = True
tau = 1
num_envs = 1

In [8]:
def train(env_id,timesteps,learning_rate,buffer_size,gamma,
          target_network_frequency,max_grad_norm,batch_size,
          start_e,end_e,exploration_fraction,learning_starts,
          train_frequency,seed,tau,num_envs):

    run_name = f"{env_id}__{seed}__{int(time.time())}"
    wandb.init(
        project="dqn-classic-control-benchmark",
        config={
            "env":env_id,
            "timesteps":timesteps,
            "lr":learning_rate,
            "buffer_size":buffer_size,
            "gamma":gamma,
            "target_network_frequency":target_network_frequency,
            "batch_size":batch_size,
            "start_e":start_e,
            "end_e":end_e,
            "exploration_fraction":exploration_fraction,
            "learning_starts":learning_starts,
            "train_frequency":train_frequency,
            "seed":seed,
            "tau":tau,
            "num_envs":num_envs,
        },
        sync_tensorboard=True,
        monitor_gym=True,
        name=run_name,
    )
    
    writer = SummaryWriter(f"runs/{run_name}")

    random.seed(seed)
    np.random.seed(seed)
    key = jax.random.PRNGKey(seed)
    key, q_key = jax.random.split(key, 2)

    envs = gym.vector.SyncVectorEnv(
        [make_env(env_id, seed + i, i, True, run_name) for i in range(num_envs)]
    )

    obs, _ = envs.reset(seed=seed)

    q_network = QNetwork(action_dim=envs.single_action_space.n)

    q_state = TrainState.create(
        apply_fn=q_network.apply,
        params=q_network.init(q_key, obs),
        target_params=q_network.init(q_key,obs),
        tx=optax.adam(learning_rate=learning_rate)
    )

    q_network.apply = jax.jit(q_network.apply)
    q_state = q_state.replace(target_params=optax.incremental_update(q_state.params, q_state.target_params, 1))

    rb = ReplayBuffer(
        buffer_size,
        envs.single_observation_space,
        envs.single_action_space,
        "cpu",
        n_envs=num_envs,
        handle_timeout_termination=False,
    )

    @jax.jit
    def update(q_state, observations, actions, next_observations, rewards, dones):
        q_next_target = q_network.apply(q_state.target_params, next_observations)
        q_next_target = jnp.max(q_next_target, axis=-1)
        next_q_value = rewards + (1 - dones) * gamma * q_next_target

        def mse_loss(params):
            q_pred = q_network.apply(params, observations)
            q_pred = q_pred[jnp.arange(q_pred.shape[0]), actions.squeeze()]
            return ((q_pred - next_q_value)**2).mean(), q_pred

        (loss_value,q_pred), grads = jax.value_and_grad(mse_loss,has_aux=True)(q_state.params)
        q_state = q_state.apply_gradients(grads=grads)
        return loss_value, q_pred, q_state

    start_time = time.time()

    obs, _ = envs.reset(seed=seed)
    for global_step in range(timesteps):
        epsilon = linear_schedule(start_e, end_e, exploration_fraction * timesteps, global_step)
        if random.random() < epsilon:
            actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
        else:
            q_values = q_network.apply(q_state.params, obs)
            actions = q_values.argmax(axis=-1)
            actions = jax.device_get(actions)

        next_obs, rewards, terminated, truncated, infos = envs.step(actions)

        if "final_info" in infos:
            for info in infos["final_info"]:
                if "episode" not in info:
                    continue

                print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                writer.add_scalar("charts/episodic_return",info["episode"]["r"], global_step)
                writer.add_scalar("charts/episodic_length",info["episode"]["l"], global_step)
                writer.add_scalar("charts/epsilon",epsilon,global_step)


        real_next_obs = next_obs.copy()
        for idx,d in enumerate(truncated):
            if d and "final_observations" in infos:
                real_next_obs[idx] = infos["final_observations"][idx]
        rb.add(obs, real_next_obs, actions, rewards, terminated, infos)

        obs = next_obs

        if global_step > learning_starts:
            if global_step % train_frequency == 0:
                data = rb.sample(batch_size)

                loss, old_val, q_state = update(
                    q_state,
                    data.observations.numpy(),
                    data.actions.numpy(),
                    data.next_observations.numpy(),
                    data.rewards.flatten().numpy(),
                    data.dones.flatten().numpy(),
                )

                if global_step % 100 == 0:
                    writer.add_scalar("losses/td_loss",jax.device_get(loss),global_step)
                    writer.add_scalar("losses/q_values",jax.device_get(old_val).mean(), global_step)
                    writer.add_scalar("charts/SPS",int(global_step / (time.time() - start_time)),global_step)

            if global_step % target_network_frequency == 0:
                q_state = q_state.replace(
                    target_params=optax.incremental_update(q_state.params, q_state.target_params, tau)
                )

    model_path = f"runs/{run_name}.cleanrl_model"
    with open(model_path,"wb") as f:
        f.write(flax.serialization.to_bytes(q_state.params))

    print("Model Saved")

    episodic_returns = evaluate(
        model_path,
        make_env,
        env_id,
        run_name=f"{run_name}-eval",
        Model=QNetwork,
        epsilon=0.05,
        eval_episodes=100
    )

    for idx, episodic_return in enumerate(episodic_returns):
        writer.add_scalar("eval/episodic_return", episodic_return,idx)

    envs.close()
    writer.close()
    wandb.finish()

In [13]:
train(envs["mountainCar"],n_timesteps,learning_rate,buffer_size,gamma,target_network_frequency,max_grad_norm,batch_size,start_e,end_e,exploration_fraction,learning_starts,train_frequency,seed,tau,num_envs)

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Moviepy - Building video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-0.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-0.mp4
global_step=199, episodic_return=[-200.]
Moviepy - Building video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-1.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-1.mp4
global_step=399, episodic_return=[-200.]
global_step=599, episodic_return=[-200.]
global_step=799, episodic_return=[-200.]
global_step=999, episodic_return=[-200.]
global_step=1199, episodic_return=[-200.]
global_step=1399, episodic_return=[-200.]
global_step=1599, episodic_return=[-200.]
Moviepy - Building video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-video-episode-8.mp4
global_step=1799, episodic_return=[-200.]
global_step=1999, episodic_return=[-200.]
global_step=2199, episodic_return=[-200.]
global_step=2399, episodic_return=[-200.]
global_step=2599, episodic_return=[-200.]
global_step=2799, episodic_return=[-200.]
global_step=2999, episodic_return=[-200.]
global_step=3199, episodic_return=[-200.]
global_step=3399, episodic_return=[-200.]
global_step=3599, episodic_return=[-200.]
global_step=3799, episodic_return=[-200.]
global_step=3999, episodic_return=[-200.]
global_step=4199, episodic_return=[-200.]
global_step=4399, episodic_return=[-200.]
global_step=4599, episodic_return=[-200.]
global_step=4799, episodic_return=[-200.]
global_step=4999, episodic_return=[-200.]
global_step=5199, episodic_return=[-200.]
global_step=5399, episodic_return=[-200.]
Moviepy - Building video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298/rl-

...

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episode-1.mp4
eval_episode=1, episodic_return=[-200.]
eval_episode=2, episodic_return=[-200.]
eval_episode=3, episodic_return=[-200.]
eval_episode=4, episodic_return=[-200.]
eval_episode=5, episodic_return=[-200.]
eval_episode=6, episodic_return=[-200.]
eval_episode=7, episodic_return=[-200.]
Moviepy - Building video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episode-8.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episode-8.mp4
eval_episode=8, episodic_return=[-200.]
eval_episode=9, episodic_return=[-200.]
eval_episode=10, episodic_return=[-200.]
eval_episode=11, episodic_return=[-200.]
eval_episode=12, episodic_return=[-200.]
eval_episode=13, episodic_return=[-200.]
eval_episode=14, episodic_return=[-200.]
eval_episode=15, episodic_return=[-200.]
eval_episode=16, episodic_return=[-200.]
eval_episode=17, episodic_return=[-200.]
eval_episode=18, episodic_return=[-200.]
eval_episode=19, episodic_return=[-200.]
eval_episode=20, episodic_return=[-200.]
eval_episode=21, episodic_return=[-200.]
eval_episode=22, episodic_return=[-200.]
eval_episode=23, episodic_return=[-200.]
eval_episode=24, episodic_return=[-200.]
eval_episode=25, episodic_return=[-200.]
eval_episode=26, episodic_return=[-200.]
Moviepy - Building video /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episo

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episode-27.mp4
eval_episode=27, episodic_return=[-200.]
eval_episode=28, episodic_return=[-200.]
eval_episode=29, episodic_return=[-200.]
eval_episode=30, episodic_return=[-200.]
eval_episode=31, episodic_return=[-200.]
eval_episode=32, episodic_return=[-200.]
eval_episode=33, episodic_return=[-200.]
eval_episode=34, episodic_return=[-200.]
eval_episode=35, episodic_return=[-200.]
eval_episode=36, episodic_return=[-200.]
eval_episode=37, episodic_return=[-200.]
eval_episode=38, episodic_return=[-200.]
eval_episode=39, episodic_return=[-200.]
eval_episode=40, episodic_return=[-200.]
eval_episode=41, episodic_return=[-200.]
eval_episode=42, episodic_return=[-200.]
eval_episode=43, episodic_return=[-200.]
eval_episode=44, episodic_return=[-200.]
eval_episode=45, episodic_return=[-200.]
eval_episode=46, episodic_return=[-200.]
eval_episode=47, episodic_return=[-200.]
eval_episode=48

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/video/MountainCar-v0__1__1699891298-eval/rl-video-episode-64.mp4
eval_episode=64, episodic_return=[-200.]
eval_episode=65, episodic_return=[-200.]
eval_episode=66, episodic_return=[-200.]
eval_episode=67, episodic_return=[-200.]
eval_episode=68, episodic_return=[-200.]
eval_episode=69, episodic_return=[-200.]
eval_episode=70, episodic_return=[-200.]
eval_episode=71, episodic_return=[-200.]
eval_episode=72, episodic_return=[-200.]
eval_episode=73, episodic_return=[-200.]
eval_episode=74, episodic_return=[-200.]
eval_episode=75, episodic_return=[-200.]
eval_episode=76, episodic_return=[-200.]
eval_episode=77, episodic_return=[-200.]
eval_episode=78, episodic_return=[-200.]
eval_episode=79, episodic_return=[-200.]
eval_episode=80, episodic_return=[-200.]
eval_episode=81, episodic_return=[-200.]
eval_episode=82, episodic_return=[-200.]
eval_episode=83, episodic_return=[-200.]
eval_episode=84, episodic_return=[-200.]
eval_episode=85

charts/SPS,▄█▇▆▆▅▅▄▃▃▂▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
charts/episodic_length,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
charts/episodic_return,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
charts/epsilon,██▇▆▆▅▅▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/episodic_return,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
losses/q_values,█▇▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/td_loss,▁▄▄▃▄▃▂█▃▄▄▃▂▃▄▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▁▁▁▂
charts/SPS,924.0
charts/episodic_length,171.0
charts/episodic_return,-171.0
